In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import streamlit as st
import pickle
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [23]:
df= pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [25]:
le= LabelEncoder()
df['Gender'] = le.fit_transform(df['Gender'])
df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=True)


In [26]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [27]:
#OHE
from sklearn.preprocessing import OneHotEncoder
ohe_ndarray = OneHotEncoder(sparse_output=False)
geo_encoded = ohe_ndarray.fit_transform(df[['Geography']])
geo_encoded

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [7]:
# Convert to sparse matrix= true whwre column names are present 
# ohe = OneHotEncoder()
# geo_encoded_name = ohe.fit_transform(df[['Geography']])
# ohe.get_feature_names_out(["Geography"])

In [28]:
geo_encoded_df=pd.DataFrame(geo_encoded, columns=ohe_ndarray.get_feature_names_out(["Geography"]))

In [29]:
df[["Geography"]].head()

,Geography
0,France
1,Spain
2,France
3,France
4,Spain


In [30]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [31]:
## combine the encoded geography with the original dataframe
df = pd.concat([df.drop("Geography",axis=1), geo_encoded_df], axis=1)

In [32]:
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [33]:
#save encoder data 
pickle.dump(ohe_ndarray, open('ohe_ndarray.pkl', 'wb'))
#save label encoder data
pickle.dump(le, open('le.pkl', 'wb'))

In [34]:
# divide the dataset into features and target variable
X = df.drop(['Exited'], axis=1)
y = df['Exited']    
# split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [35]:
## scale the features
scalar= StandardScaler()
X_train = scalar.fit_transform(X_train)
X_test = scalar.transform(X_test)

In [39]:

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((8000, 12), (2000, 12), (8000,), (2000,))

In [40]:
# save the scaler
pickle.dump(scalar, open('scalar.pkl', 'wb'))

In [41]:
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [42]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime



In [45]:
X_train.shape[1]

12

In [ ]:

### Define the model ANN
model = Sequential(
    [
        Dense(64, activation='relu', input_shape=(X_train.shape[1],)),##HL1 connected with input layer (X_train.shape[1]) this only present in first hidden layer
        Dense(32, activation='relu'), ##HL2
        Dense(16, activation='relu'),##HL2
        Dense(1, activation='sigmoid')  # Output layer for binary classification
         

    ]
)